In [ ]:
from dotenv import load_dotenv
import os

# Automatically find and load .env in the project root
load_dotenv()

# Now your key is in the environment
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY not set in environment")

# For the OpenAI Python SDK (if you use it directly)
import openai
openai.api_key = api_key



In [7]:
import chromadb
from chromadb.config import Settings

# 1. Create a Settings object with *only* the persist_directory
settings = Settings(persist_directory="./chroma_db")

# 2. Instantiate the client with those settings
client = chromadb.Client(settings=settings)

# 3. Create (or fetch) your collection as before
collection = client.get_or_create_collection(name="food_images")



In [8]:
from langchain.embeddings import HuggingFaceEmbeddings

# 1. Use a lightweight, all-in-one model
embedder = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# 2. Your sample data
docs    = [
    "Apple: 95 kcal, 0.3 g fat, 0.5 g protein",
    "Banana: 105 kcal, 0.4 g fat, 1.3 g protein"
]
metas   = [{"food": "apple"}, {"food": "banana"}]
doc_ids = ["apple_1", "banana_1"]

# 3. Embed & add to Chroma
embs = embedder.embed_documents(docs)
collection.add(
    documents=docs,
    embeddings=embs,
    metadatas=metas,
    ids=doc_ids,
)



In [9]:
# Query for “banana”
query = "banana"

# 1. Embed the query
query_emb = embedder.embed_query(query)

# 2. Query Chroma for the top match
result = collection.query(
    query_embeddings=[query_emb],
    n_results=1,
)

# 3. Inspect the result
print(result)


{'ids': [['banana_1']], 'embeddings': None, 'documents': [['Banana: 105 kcal, 0.4 g fat, 1.3 g protein']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'food': 'banana'}]], 'distances': [[0.8133772611618042]]}


In [11]:
from langchain.vectorstores import Chroma as LCChroma
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI  # or any other LLM

# 1. Point LangChain at your existing Chroma store
vectorstore = LCChroma(
    persist_directory="./chroma_db",
    collection_name="food_images",
    embedding_function=embedder,
)

# 2. Build a retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

# 3. Create a RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),       
    retriever=retriever,
)

# 4. Ask a question
answer = qa_chain.run("How many calories are in a banana?")
print(answer)


 There are about 105 calories in a medium banana.


In [13]:
from fastapi import FastAPI
from pydantic import BaseModel
import sqlite3
from contextlib import asynccontextmanager

# Define lifespan handler for startup and shutdown events
@asynccontextmanager
async def lifespan(app: FastAPI):
    # Startup: create SQLite database and table
    conn = sqlite3.connect("nutrition.db")
    cursor = conn.cursor()
    cursor.execute(
        """
        CREATE TABLE IF NOT EXISTS foods (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            calories INTEGER NOT NULL
        )
        """
    )
    conn.commit()
    conn.close()
    yield
    # Shutdown: nothing to clean up for SQLite

# Pass lifespan to FastAPI
app = FastAPI(lifespan=lifespan)

# Data model for demonstration
class FoodItem(BaseModel):
    id: int | None = None
    name: str
    calories: int

@app.get("/")
def read_root():
    return {"message": "Hello, Nutrition API!"}

@app.post("/foods/")
def create_food(item: FoodItem):
    conn = sqlite3.connect("nutrition.db")
    cursor = conn.cursor()
    cursor.execute(
        "INSERT INTO foods (name, calories) VALUES (?, ?) RETURNING id",
        (item.name, item.calories)
    )
    row = cursor.fetchone()
    conn.commit()
    conn.close()
    return {"id": row[0], "name": item.name, "calories": item.calories}

@app.get("/foods/{food_id}")
def read_food(food_id: int):
    conn = sqlite3.connect("nutrition.db")
    cursor = conn.cursor()
    cursor.execute(
        "SELECT id, name, calories FROM foods WHERE id = ?",
        (food_id,)
    )
    row = cursor.fetchone()
    conn.close()
    if row:
        return {"id": row[0], "name": row[1], "calories": row[2]}
    return {"error": "Food not found"}

# To run:
# uvicorn main:app --reload --port 8000

